In [1]:
import os
import time
from dotenv import load_dotenv
from pymongo import MongoClient, ASCENDING

In [3]:
load_dotenv()

MONGO_URI = os.environ["MONGO_URL"]
DB_NAME = os.environ.get("MONGO_DB", "appdb")
COLL_NAME = os.environ.get("MONGO_COLLECTION", "documents")
EMBED_DIM = int(os.environ.get("EMBED_DIM", "1536"))

In [8]:
import os
import time
from dotenv import load_dotenv
from pymongo import MongoClient, ASCENDING
from pymongo.operations import SearchIndexModel

load_dotenv()

MONGO_URI = os.environ["MONGO_URL"]
DB_NAME = os.environ.get("MONGO_DB", "appdb")
COLL_NAME = os.environ.get("MONGO_COLLECTION", "documents")
EMBED_DIM = int(os.environ.get("EMBED_DIM", "1536"))
VECTOR_INDEX_NAME = os.environ.get("VECTOR_INDEX_NAME", "vector_index")


def wait_for_mongo(client: MongoClient, timeout_s: int = 60) -> None:
    deadline = time.time() + timeout_s
    last_err = None
    while time.time() < deadline:
        try:
            client.admin.command("ping")
            return
        except Exception as e:
            last_err = e
            time.sleep(1)
    raise RuntimeError(f"Mongo not ready after {timeout_s}s: {last_err}")


def ensure_basic_indexes(coll) -> None:
    # Example: unique ID + a common filter index (normal MongoDB indexes)
    coll.create_index([("doc_id", ASCENDING)], unique=True, name="doc_id_unique")
    coll.create_index([("source", ASCENDING)], name="source_idx")


def ensure_vector_search_index(coll, index_name: str) -> None:
    """
    Creates or updates an Atlas Vector Search index using PyMongo helpers.
    This requires an Atlas Local deployment (mongodb/mongodb-atlas-local) or Atlas.
    """
    model = SearchIndexModel(
        definition={
            "fields": [
                {
                    "type": "vector",
                    "path": "embedding",
                    "numDimensions": EMBED_DIM,
                    "similarity": "cosine",
                    # Optional (often helpful for speed/memory):
                    # "quantization": "scalar",
                },
                # Optional: add filterable fields for hybrid search
                # (Only include these if you plan to filter on them.)
                # {"type": "filter", "path": "source"},
                # {"type": "filter", "path": "doc_id"},
            ]
        },
        name=index_name,
        type="vectorSearch",
    )

    # Check if it already exists
    existing = list(coll.list_search_indexes())
    exists = any(idx.get("name") == index_name for idx in existing)

    if exists:
        # Update existing definition (keeps the same index name)
        coll.update_search_index(name=index_name, definition=model.document["definition"])
        print(f"Updated vector search index: {index_name}")
    else:
        created_name = coll.create_search_index(model=model)
        print(f"Created vector search index: {created_name} (building...)")

    # Poll until queryable
    print("Polling until the vector index is queryable...")
    deadline = time.time() + 120
    while time.time() < deadline:
        idx = list(coll.list_search_indexes(index_name))
        if idx and idx[0].get("queryable") is True:
            print(f"Vector index '{index_name}' is ready.")
            return
        time.sleep(2)

    raise TimeoutError(f"Vector index '{index_name}' not queryable after 120s.")


def main() -> None:
    client = MongoClient(MONGO_URI)
    wait_for_mongo(client)

    db = client[DB_NAME]
    coll = db[COLL_NAME]

    ensure_basic_indexes(coll)
    ensure_vector_search_index(coll, VECTOR_INDEX_NAME)

    print("Bootstrap complete.")
    client.close()


if __name__ == "__main__":
    main()


Created vector search index: vector_index (building...)
Polling until the vector index is queryable...
Vector index 'vector_index' is ready.
Bootstrap complete.


In [ ]:
import random


client = MongoClient(MONGO_URI)
coll = client[DB_NAME][COLL_NAME]

def random_embedding(dim: int):
    # Replace this with real embeddings from your model
    return [random.uniform(-1, 1) for _ in range(dim)]

docs = [
    {
        "doc_id": "doc-10",
        "text": "MongoDB Atlas Vector Search is fast.",
        "source": "demo",
        "embedding": random_embedding(EMBED_DIM),
    },
    {
        "doc_id": "doc-20",
        "text": "Vector databases enable semantic search.",
        "source": "demo",
        "embedding": random_embedding(EMBED_DIM),
    },
    {
        "doc_id": "doc-30",
        "text": "Time travel movies are fun.",
        "source": "demo",
        "embedding": random_embedding(EMBED_DIM),
    },
]

coll.insert_many(docs)
print("Inserted documents.")
client.close()


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: project_altitude.documents index: doc_id_unique dup key: { doc_id: "doc-1" }', 'keyPattern': {'doc_id': 1}, 'keyValue': {'doc_id': 'doc-1'}, 'op': {'doc_id': 'doc-1', 'text': 'MongoDB Atlas Vector Search is fast.', 'source': 'demo', 'embedding': [0.8491375772905148, -0.3071353334634441, 0.18958303881204142, -0.47536606917685775, -0.2857914206029204, -0.18191267959612434, 0.3306390338040104, -0.35944901072195923, 0.070431424660089, 0.8319784608162084, -0.5728362103157512, 0.20443980606754542, -0.9700732019271641, 0.39424942552527753, 0.5981688424076583, -0.5544159046336881, 0.4664970114858089, -0.9434921769768296, 0.9881074705948092, 0.13911085959847713, 0.08479326208182347, 0.7483837802618591, -0.33267445403675544, 0.3994203271563521, -0.4777307641172037, -0.6110008375435894, 0.4126621273040345, 0.18885784145097828, 0.6000693930393948, 0.4

In [ ]:
client = MongoClient(MONGO_URI)
coll = client[DB_NAME][COLL_NAME]

def query_embedding(dim: int):
    # Replace with your real query embedding
    return [random.uniform(-1, 1) for _ in range(dim)]

pipeline = [
    {
        "$vectorSearch": {
            "index": "vector_index",
            "path": "embedding",
            "queryVector": query_embedding(EMBED_DIM),
            "numCandidates": 100,
            "limit": 5,
            # Optional metadata filtering:
            # "filter": { "source": { "$eq": "demo" } },
        }
    },
    {
        "$project": {
            "_id": 0,
            "doc_id": 1,
            "text": 1,
            "score": {"$meta": "vectorSearchScore"},
        }
    },
]

results = coll.aggregate(pipeline)
for r in results:
    print(r)

client.close()

{'doc_id': 'doc-3', 'text': 'Time travel movies are fun.', 'score': 0.5247224569320679}
{'doc_id': 'doc-2', 'text': 'Vector databases enable semantic search.', 'score': 0.48832210898399353}
{'doc_id': 'doc-1', 'text': 'MongoDB Atlas Vector Search is fast.', 'score': 0.48614275455474854}


In [20]:
import os
import time
from dotenv import load_dotenv
from pymongo import MongoClient, ASCENDING
from pymongo.operations import SearchIndexModel

load_dotenv()

MONGO_URI = os.environ["MONGO_URL"]
DB_NAME = os.environ.get("MONGO_DB", "appdb")
EMBED_DIM = int(os.environ.get("EMBED_DIM", "1536"))



client = MongoClient(MONGO_URI)
db = client[DB_NAME]

# Create User collection
users = db["users"]
users.create_index("user_id", unique=True)
users.create_index([("username", ASCENDING)], unique=True)
users.create_index([("email", ASCENDING)], unique=True)

# create Contact collection
contacts = db["contacts"]
contacts.create_index([("contact_id", ASCENDING)], unique=True)
contacts.create_index([("owner_user_id", ASCENDING)])

# Create Contact Note collection
contact_notes = db["contact_notes"]
contact_notes.create_index([("note_id", ASCENDING)], unique=True)
contact_notes.create_index([("user_id", ASCENDING)])
contact_notes.create_index([("contact_id", ASCENDING)])
contact_notes.create_index([("user_id", ASCENDING), ("label", ASCENDING)], name="by_user_label")

# Create Contact Note Vector Embedding vector search index
index_name = "contact_note_vector_index"
model = SearchIndexModel(
    name=index_name,
    type="vectorSearch",
    definition={
        "fields":
            [
                {
                   "type": "vector",
                   "path":  "embedding",
                   "numDimensions": EMBED_DIM,
                   "similarity": "dotProduct",
                   "quantization": "scalar"
                },
                {
                    "type": "filter",
                    "path": "user_id"
                },
                {
                    "type": "filter",
                    "path": "contact_id"
                }
            ]
    }
)

result = contact_notes.create_search_index(model=model)
print("Polling...")
predicate = None
if predicate is None:
    predicate = lambda index: index.get("queryable") is True

while True:
    indices = list(contact_notes.list_search_indexes(result))
    if len(indices) and predicate(indices[0]):
        break
print(f"{result} is ready")
client.close()








Polling...
contact_note_vector_index is ready


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', directconnection=True), 'project_altitude'), 'users')